# 인체 측정데이터 분석

* 거동이 불편한 환자의 신장 몸무게 등을 측정해야하는 경우가 있다. 신체의 일부만으로 신장이나 몸무게를 예측할 수 있다면 이와 같은 환자에게 큰 도움이 될 것이다.
* 신체 중에서 어떤 부위의 조합을 학습시키는 것이 신장이나 몸무게를 가장 잘 예측하는지 데이터를 기반으로 하여 분석해 보려고 한다.
* 신체를 여러 부분으로 나누어 여러 모델에 적용시켜 본다.

# 1. Loading Data

In [1]:
import pandas as pd
data1_male = pd.read_excel('2004_5차_직접측정 데이터.xlsx')
data1_female = pd.read_excel('2004_5차_직접측정 데이터.xlsx', sheet_name='여성')
data2_male = pd.read_excel('2010_6차_직접측정 데이터.xlsx')
data2_female = pd.read_excel('2010_6차_직접측정 데이터.xlsx', sheet_name='여성')

In [2]:
data1 = data1_male.append(data1_female, ignore_index=True)
data2 = data2_male.append(data2_female, ignore_index=True)
data3 = pd.read_excel('2015_7차_직접측정 데이터.xlsx')

In [6]:
est_hand_5 = data1[['성별', '(102)키', '(132)몸무게', '(208)가슴둘레', '(211)허리둘레', '(213)엉덩이둘레', '(218)장딴지둘레']]

In [7]:
est_hand_6 = data2[['성별', '104:키', '510:몸무게', '208:가슴둘레', '211:허리둘레', '214:엉덩이둘레', '236:겨드랑둘레', '423:장딴지둘레', '425:발목최대둘레', '109:겨드랑높이', '111:엉덩이높이', '113:허리높이', '116:무릎높이']]


In [8]:
est_hand_7 = data3[['ⓞ_02_성별', '①_003_키', '①_031_몸무게', '①_039_가슴둘레', '①_042_허리둘레', '①_045_엉덩이둘레', '①_067_겨드랑둘레', '①_123_무릎둘레', '①_125_장딴지둘레', '①_127_발목최대둘레', '①_008_겨드랑높이', '①_011_엉덩이높이', '①_013_허리높이', '①_016_무릎높이']]


In [9]:
est_hand_5.columns

Index(['성별', '(102)키', '(132)몸무게', '(208)가슴둘레', '(211)허리둘레', '(213)엉덩이둘레',
       '(218)장딴지둘레'],
      dtype='object')

In [10]:
est_hand_6.columns

Index(['성별', '104:키', '510:몸무게', '208:가슴둘레', '211:허리둘레', '214:엉덩이둘레',
       '236:겨드랑둘레', '423:장딴지둘레', '425:발목최대둘레', '109:겨드랑높이', '111:엉덩이높이',
       '113:허리높이', '116:무릎높이'],
      dtype='object')

In [11]:
est_hand_7.columns

Index(['ⓞ_02_성별', '①_003_키', '①_031_몸무게', '①_039_가슴둘레', '①_042_허리둘레',
       '①_045_엉덩이둘레', '①_067_겨드랑둘레', '①_123_무릎둘레', '①_125_장딴지둘레',
       '①_127_발목최대둘레', '①_008_겨드랑높이', '①_011_엉덩이높이', '①_013_허리높이',
       '①_016_무릎높이'],
      dtype='object')

### - Summarize 1
> 2015년 7차 측정데이터가 가장 최신 데이터이기도 하고, 2개를 제외하고 모든 feature를 포함하고 있으므로 먼저 이 dataset을 중심으로 여러모델에 학습을 시켜본다. (Linear Regression, Random Forest, SVM)

> 만약 다른 년도의 데이터를 함께 본다면 NULL값이 생기는 것을 부담하면서까지 포함 시킬 필요가 있을지 생각해 보아야 할 것 같다. 측정 단위 또한 다르다.

In [97]:
data = est_hand_7.copy()
data.columns = ['성별', '키', '몸무게', '가슴둘레', '허리둘레', '엉덩이둘레', '겨드랑둘레', '무릎둘레', '장딴지둘레', '발목최대둘레', '겨드랑높이', '엉덩이높이', '허리높이', '무릎높이']
dataset = data.copy()

# nan값을 가진 행이 5개정도 뿐이어서 일단 삭제해 주었다.
dataset.dropna(inplace=True)

data_height = dataset[['성별', '키', '겨드랑높이', '엉덩이높이', '허리높이', '무릎높이']]
data_weight = dataset[['성별', '몸무게', '가슴둘레', '허리둘레', '엉덩이둘레', '겨드랑둘레', '장딴지둘레']]

# 2. Correlation
> 남성 / 여성을 나누지 않은 전체적인 상관관계

In [98]:
def corr(dataset, feature):
    corr_matrix = dataset.corr()
    return corr_matrix[feature].sort_values(ascending=False)

In [99]:
corr(data_height, '키')

키        1.000000
겨드랑높이    0.976460
허리높이     0.955721
엉덩이높이    0.910349
무릎높이     0.860201
Name: 키, dtype: float64

In [100]:
corr(data_weight, '몸무게')

몸무게      1.000000
가슴둘레     0.906537
장딴지둘레    0.865352
엉덩이둘레    0.834552
허리둘레     0.833738
겨드랑둘레    0.826862
Name: 몸무게, dtype: float64

# 3. Data Preprocessing
> DataFrameSelector

In [101]:
from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, h_or_w, m_or_w):
        self.h_or_w = h_or_w
        self.m_or_w = m_or_w
        self.col_height = ['성별', '키', '겨드랑높이', '엉덩이높이', '허리높이', '무릎높이']
        self.col_weight = ['성별', '몸무게', '가슴둘레', '허리둘레', '엉덩이둘레', '겨드랑둘레', '장딴지둘레']
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        
        if self.h_or_w == 'height':
            df = X[self.col_height]
            result = df[df['성별'] == self.m_or_w]

        elif self.h_or_w == 'weight':
            df = X[self.col_weight]
            result = df[df['성별'] == self.m_or_w]
          
        else:
            print('ERROR! check your parameters')
            return
        
        result_X, result_y = result[result.columns[2:]], result[result.columns[1]]
        return result, result_X, result_y

In [158]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelBinarizer

def purpose(weight_or_height, gender):
    data_selector = DataFrameSelector(weight_or_height, gender)
    selected = data_selector.fit_transform(dataset)
    full = selected[0]
    X = selected[1]
    y = selected[2]
    return full, X, y

data_selected = purpose('height', '남')
std = StandardScaler()

prepared_X = std.fit_transform(data_selected[1])
prepared_y = data_selected[2]

# 4. Linear Regression

In [159]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(prepared_X, prepared_y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

> Training MSE

In [160]:
from sklearn.metrics import mean_squared_error
import numpy as np

predicted_y = lin_reg.predict(prepared_X)
lin_mse = mean_squared_error(prepared_y, predicted_y)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

16.729137732677344

In [161]:
some_X = prepared_X[:5]
some_y = prepared_y[:5]
some_predicted = lin_reg.predict(some_X)
for some_y, some_predicted in zip(some_y, some_predicted):
    print('실제 값: ', some_y, '예측 값: ', some_predicted)

실제 값:  1736.0 예측 값:  1717.7110194807065
실제 값:  1833.0 예측 값:  1815.5753418950535
실제 값:  1744.0 예측 값:  1725.8506688328266
실제 값:  1751.0 예측 값:  1726.8837503774464
실제 값:  1851.0 예측 값:  1849.3992522445355


# 5. Cross_Validation
> Test MSE

In [162]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(lin_reg, prepared_X, prepared_y, scoring='neg_mean_squared_error', cv=10)
lin_rmse_scores = np.sqrt(-scores)

In [163]:
def display_scores(scores):
    print('Scores:', scores)
    print('Mean:', scores.mean())
    print('Standard deviation:', scores.std())

In [164]:
display_scores(lin_rmse_scores)

Scores: [17.06870819 16.70898524 14.03773381 13.89190545 16.92558124 17.59896312
 17.82199616 17.19651789 19.30464865 18.26739122]
Mean: 16.882243095861828
Standard deviation: 1.6242627271532586


In [165]:
from sklearn.model_selection import cross_val_predict

predicted_y_c = cross_val_predict(lin_reg, prepared_X, prepared_y, cv=10)

In [166]:
count = 0
for prepared_y, predicted_y_c in zip(prepared_y, predicted_y_c):
    print('실제 값: ', prepared_y, '예측 값: ', predicted_y_c)
    count += 1
    if count == 5:
        break

실제 값:  1736.0 예측 값:  1717.2721261175186
실제 값:  1833.0 예측 값:  1814.6294583739902
실제 값:  1744.0 예측 값:  1725.3027292400843
실제 값:  1751.0 예측 값:  1726.2655262843384
실제 값:  1851.0 예측 값:  1848.5211640672549


### summarize 2
> linear Regression의 결과 적은 비교적 적은 Test MSE를 보였다. Feature Engineering을 할 때 StandardScaler를 이용하여 정규화를 해주었는데, 하지 않았을 때와 결과가 같았다.

> 다음은 여러가지 모델들에 여러가지 데이터 조합으로 그 성능을 비교해보고 결과를 시각화 해보려고 한다.

# 6. Other Models

In [169]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings('ignore')

height_man_data = purpose('height', '남')
height_woman_data = purpose('height', '여')
weight_man_data = purpose('weight', '남')
weight_woman_data = purpose('weight', '여')

lin_reg = LinearRegression()
forest_reg = RandomForestRegressor()
svm_reg = svm.SVR()

data_list = [height_man_data, height_woman_data, weight_man_data, weight_woman_data]
model_list = [lin_reg, forest_reg, svm_reg]

rmse_matrix = []
for data in data_list:
    score_list = []
    for model in model_list:
        scores = cross_val_score(model, data[1], data[2], scoring='neg_mean_squared_error', cv=10)
        rmse_score = np.sqrt(-scores).mean()
        score_list.append(rmse_score)
    rmse_matrix.append(score_list)
rmse_matrix

[[16.88224309586183, 18.890008970126683, 59.45432079290909],
 [14.634858437874687, 16.317083909210126, 57.880794873841026],
 [2.990873800980138, 3.194005205457321, 11.856205685961706],
 [2.2288388794984284, 2.3501766757070053, 8.810407985412656]]

In [186]:
model_result = pd.DataFrame(rmse_matrix, columns=['lin_reg', 'forest_reg', 'svm_reg'], index=['height_man', 'height_woman', 'weight_man', 'weight_woman'])
model_result

,lin_reg,forest_reg,svm_reg
height_man,16.882243,18.890009,59.454321
height_woman,14.634858,16.317084,57.880795
weight_man,2.990874,3.194005,11.856206
weight_woman,2.228839,2.350177,8.810408
